# Function Calling
This demonstration build upon demonstrations built by Mark Whitby and documented at [Introduction to Intelligent Apps](https://github.com/Azure/intro-to-intelligent-apps). This is a great source for learning Azure OpenAI step by step.

In [ ]:
import os
import openai
from openai import OpenAI
import json
from dotenv import load_dotenv


# Load environment variables
if load_dotenv():
    print("Found OpenAPI Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("No file .env found")

# Setting up the deployment name
deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") 

# Currently Chat Completion API have the following versions available: 2023-07-01-preview
openai.api_version = os.getenv("OPENAI_API_VERSION") 

from openai import AzureOpenAI
 
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2023-12-01-preview",
  azure_deployment = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
)

Test that we can connect to OpenAI instance and do some basic chat

In [ ]:
response = client.chat.completions.create(
    model = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME"),
    messages = [{"role" : "assistant", "content" : "The one thing I love more than anything else is "}],
)

print(response)

Now setup a function that we can later call

In [ ]:
import requests
endpoint = os.getenv("SERVICE_NOW_ENDPOINT")
print(endpoint)
user = os.getenv("SERVICE_NOW_USER")
password = os.getenv("SERVICE_NOW_PASSWORD")

def get_incident_details(incident_number):
    """
    Fetches details of a specific incident from ServiceNow.

    Parameters:
    - incident_number: The number of the incident to fetch.

    Returns:
    A dictionary with the incident's ID, description, status, and last update time if found, otherwise None.
    """
    # Construct the URL with the incident number as a query parameter
    url = f"{endpoint}/api/now/table/incident?number={incident_number}"
    
    # Make the request to ServiceNow
    response = requests.get(url, auth=(user, password))
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Assuming the response contains results and we take the first one
        if data['result']:
            incident = data['result'][0]
            # Extracting required details
            details = {
                'id': incident['sys_id'],
                'description': incident['short_description'],
                'status': incident['state'],
                'last_updated': incident['sys_updated_on']
            }
            return details
    else:
        print(f"Error fetching incident details: {response.status_code}")
    
    return None

Test the function alone

In [ ]:
# ServiceNow parameters

response = get_incident_details("INC0008111")
print(f"Returned {response}")

Noew set this function up as a tool for Azure OpenAI

In [10]:
# Example function hard coded to return the same weather
# In production, this could be your backend API or an external API
# not added for now to the definition of the function
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

# Define the functions to use
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_incident_details",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "incident_number": {
                        "type": "string",
                        "description": "the incident number to fetch",
                    },
                    "unit": {"type": "string"},
                },
                "required": ["incident_number"],
            },
        },
    }
]

def get_function_call(messages, tool_choice = "auto"):
    

    # Call the model with the user query (messages) and the functions defined in the functions parameter
    response = client.chat.completions.create(
        model = deployment_name,
        messages = messages,
        tools = tools,
        tool_choice = tool_choice, 
    )

    return response.choices[0].message

Call OpenAI and see what tool it suggests can be used

In [ ]:
first_message = [{"role": "user", "content": "Can you get details of incident INC0008111?"}]
# 'auto' : Let the model decide what function to call
print("Let the model decide what function to call:")

response_message = get_function_call(first_message, "auto")

print(response_message)



Now actually call the function using the suggested tool from the OpenAI response

In [ ]:
tool_calls = response_message.tool_calls
# Step 2: check if the model wanted to call a function
if tool_calls:
    print("it decided to call the following function:")
    print(tool_calls)
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_incident_details": get_incident_details,
    }  # only one function in this example, but you can have multiple
    first_message.append(response_message)  # extend conversation with assistant's reply
    # Step 4: send the info for each function call and function response to the model
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            incident_number=function_args.get("incident_number")
        )
        first_message.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )
        print(first_message)
        # extend conversation with function response
        second_response = client.chat.completions.create(
            model = deployment_name,
            messages = first_message
        )
        print("The model responds with the following message:")
        print(second_response.choices[0].message)
    # get a new response from the model where it can see the function response


# 'none' : Don't call any function 
#print("Don't call any function:")
#print(get_function_call(first_message, "none")["choices"][0]['message'])

# force a specific function call
#print("Force a specific function call:")
#print(get_function_call(first_message, function_call={"name": "get_current_weather"})["choices"][0]['message'])